In [118]:
'''
# Get input from the user
# Determine if it has an image or not
# Preprocess the input based on the above output
# Get the output 
# Determine news category
# Determine trending fraction
# Pass to enhancer
# Get one of the five categories for this input
'''
import sys
sys.path.append('drive/My Drive/Fake-News-Detection/')
sys.path.append('../')
#===============================
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
from sklearn.metrics import confusion_matrix, precision_score
import os
from datetime import date
import Cleaning # for processOnNewInput
from Cleaning.cleaning import applyLinguistics
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.models import Sequential
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
import numpy as np
import os

# GLOBAL VARIABLES:
MAX_LEN = 5000 
VOCAB_SIZE = 0
TEST_SIZE = 1
pd.set_option('display.max_columns', None)
#IMAGE_DIR = 'drive/My Drive/SingleInstanceImage/'
IMAGE_DIR = ''

# PREPROCESSING FUNCTIONS:

def text_implict_preprocessing(texts):
    # initialize tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    vocab_size = len(tokenizer.word_index) + 1

    sequences = tokenizer.texts_to_sequences(texts)

    padded_texts = pad_sequences(sequences, maxlen=MAX_LEN, padding='post')

    return tokenizer, padded_texts, vocab_size

def load_embedding_matrix(tokenizer, VOCAB_SIZE):
    embeddings_index = dict()
    f = open('drive/My Drive/glove_data/glove.6B/glove.6B.100d.txt')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    # create a weight matrix for words in training docs
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

def image_implicit_preprocessing(uuid, type):
    image_dataset = []
    for i in range(len(uuid)):
        sub_dir = ''
        path = os.path.join(IMAGE_DIR, sub_dir)
        temp_img = image.load_img(os.path.join(path, str(uuid[i])+'.jpg'),target_size=(224,224))
        temp_img = image.img_to_array(temp_img)
        image_dataset.append(temp_img)

    image_dataset = np.array(image_dataset)
    image_dataset = preprocess_input(image_dataset)

    return image_dataset

# SINGLE INSTANCE FUNCTIONS:

def getEmotions(df):
  df['emotion'], df['anger'], df['disgust'], df['fear'], df['joy'], df['sadness'], df['surprise'], df['trust'], df['anticipation'] = ['Trust', 0.038, 0.00204, 0.2370, 0.2667, 0.0022, 0.0173, 0.4206, 0.015]
  return df

def getSentiment(df):
  df['category'], df['category_probability'], df['no_of_faces'], df['image_width'], df['image_height'], df['sentence_Count'], df['neg'], df['neu'], df['pos'], df['SentimentCategory'] = ['entertainment', 0.4429, 4,768,513,86,0.053,0.865,0.083,'positive']
  return df

def hasImage(df):
  dfToCheckImage= df
  image = dfToCheckImage['main_img_url']
  print(image[0])

  if image[0] == 'NULL':
    return False
  elif image[0] != 'NULL':
    return True

def getProbability(id = 0, published = date.today(), text = "NA", title = 'notitle', language = 'english', uuid = 123456789, main_img_url = 'abc'):

  # return df with basic parameters defined

  df = pd.DataFrame()
  df = df.append({'id' : id, 'language' : language, 'main_img_url' : main_img_url, 'published' : str(published), 'text' : text, 'title' : title , 'type' : 'real', 'uuid' : uuid, }, ignore_index=True)
  df = applyLinguistics(df)
  df['capital_words'] = 0
  print(df.head())

  #return df with emotion
  df = getEmotions(df)
  print(df.head())

  #return df with sentiment
  df = getSentiment(df)
  print(df.head())


  # Check for image and call the respective workflows
  if hasImage(df):
    workflowWithImage(df)
  elif not hasImage(df):
    workflowWithoutImage(df)


def workflowWithImage(df):

  textImplicit, textExplicit, t3, t4, label = preprocessWithImage(df)
  print("Going To Predict")
  probability_TICNN = predictWithImage(textImplicit, textExplicit, t3, t4)


def workflowWithoutImage(df):
  textImplicit, textExplicit, label = preprocessWithoutImage(df)
  print("Going To Predict")
  probability_TCNN = predictWithoutImage(textImplicit, textExplicit)


def preprocessWithoutImage(df):
  # Load input dataframe
  input_df = df

  # converting dataframe to numpy array
  input_data = input_df.to_numpy()

  # extract dataset and label
  X = np.delete(input_data, 6, 1)
  Y = input_data[:, 6]

  # Encoding the label
  encoder = LabelEncoder()
  encoder.fit(Y)
  Y = encoder.transform(Y)

  X_valid = X
  Y_valid = Y

  print(X_valid.shape)

  # Split dataset into branches

  # Text explicit branch
  X_valid_text_explicit = np.concatenate((X_valid[:,7:15], X_valid[:,16:24], X_valid[:,29:33]),axis=1)
  X_train = X_valid
  tokenizer, padded_texts, VOCAB_SIZE = text_implict_preprocessing(np.array(X_valid[:,4]))
  X_valid_text_implicit = padded_texts

  print(X_valid_text_implicit.shape)

  # #Load Embedding Matrix
  embedding_matrix = load_embedding_matrix(tokenizer, VOCAB_SIZE)

  return X_valid_text_implicit, X_valid_text_explicit, Y

def preprocessWithImage(df):

  X_valid_text_implicit, X_valid_text_explicit, Y = preprocessWithoutImage(df)
  # Load input dataframe
  input_df = df

  # converting dataframe to numpy array
  input_data = input_df.to_numpy()

  # extract dataset and label
  X = np.delete(input_data, 6, 1)
  Y = input_data[:, 6]

  X_valid = X
  Y_valid = Y

  X_valid_img_explicit = X_valid[:,26:29]

  encoder = LabelEncoder()
  encoder.fit(Y)
  Y = encoder.transform(Y)
  
  # Image implict branch
  X_valid_img_implicit = image_implicit_preprocessing(X_valid[:,6], Y_valid)

  return X_valid_text_implicit, X_valid_text_explicit, X_valid_img_explicit, X_valid_img_implicit, Y



def predictWithImage(t1,t2,t3,t4):
  # NEED TO CHANGE THE MODEL NAME, REST ALL IS DONE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  # load json and create model
  json_file = open('drive/My Drive/models/textCompiledModel.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)

  # load weights into new model
  loaded_model.load_weights("drive/My Drive/models/textCompiledModelRun2.h5")
  print("Loaded model from disk")

  loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics= ['accuracy', 'mse'])

  score = loaded_model.predict([t1,t2,t3,t4])

  print(score)
  return score



def predictWithoutImage(t1, t2):
  # load json and create model
  json_file = open('drive/My Drive/models/textCompiledModel.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  loaded_model = model_from_json(loaded_model_json)

  # load weights into new model
  loaded_model.load_weights("drive/My Drive/models/textCompiledModelRun2.h5")
  print("Loaded model from disk")

  loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics= ['accuracy', 'mse'])

  score = loaded_model.predict([t1, t2])

  print(score)
  return score


  # ---------------------------

def getNewsCategory():
  pass

def getTrendingFraction():
  pass

def getDelta():
  pass

def getCategory():
  pass


if __name__ == '__main__':
  input_text = "President-elect Donald J. Trump s wife, Melania, and their 10-year-old son, Barron, will not immediately move to the White House after Mr. Trump takes office in January, out of concern for uprooting Barron in the middle of the school year, transition officials said on Sunday. Ms. Trump and Barron will remain in New York for at least another six months, the officials said. Speaking to reporters on Sunday at his golf course in Bedminster, N.J., Mr. Trump said they would join him very soon  after hes finished with school. Mr. Trump said he would move to the White House. Still, with his family remaining at Trump Tower, the likelihood may increase that Mr. Trump will spend part of the week in New York. Jason Miller, a spokesman for Mr. Trump, said, No official statement has been released by the Trump family regarding transition timing, but like any parents, they are concerned about pulling their 10-year-old son out of school in the middle of the year. Mr. Miller urged that the same privacy and security considerations given to previous first families with regard to minor children be extended to the Trumps as well. The New York Times previously reported that Ms. Trump was planning to move to the White House eventually but was concerned about disrupting her sons routine. The New York Post reported on Sunday that she and Barron would not join her husband immediately after the inauguration. For all of the enthusiasm that the president-elect displayed on the campaign trail, Ms. Trump, a former model, never shared in it. She did not join her husband other than for major events like the Republican National Convention, and she granted few interviews. She is the main caregiver for their son, who attends a private school in Manhattan and is taken there most days by his mother. She is said to be concerned not just about the potential disruption to her sons life, but also about bringing him closer to the news media spotlight. People who know her have said she plans to revisit the issue at the end of the school year. The situation is similar in ways to the one Michelle Obama faced in 2008 after her husband was elected. Mrs. Obama also did not care for the attention that came with the campaign, and she was said to be considering remaining in Chicago with their two daughters. But she ultimately made the move with their children. Mr. Trump and his aides have not said whether he plans to move to the White House full time. But he has told aides he would like to be in New York when he can. Keeping Trump Tower as a residence for the incoming presidents family will create security and logistical hurdles for the Secret Service and the Police Department. The tower is on Fifth Avenue, a major thoroughfare. A version of this article appears in print on November 21, 2016, on Page A13 of the New York edition with the headline: Melania and Barron Trump Wont Immediately Move to White House, Officials Say. Order Reprints | Today's Paper | Subscribe"
  df = getProbability(id = 0, published = date.today(), text = input_text, title = 'notitle', language = 'english', uuid = '584700e476e0d3c20731cb3d28e6ce2b', main_img_url = 'NOT NULL')


    id language main_img_url   published  \
0  0.0  english     NOT NULL  2020-04-21   

                                                text    title  type  \
0  President-elect Donald J. Trump s wife, Melani...  notitle  real   

                               uuid  word_count  average_word_count  \
0  584700e476e0d3c20731cb3d28e6ce2b         518                14.0   

   exclamation_count  capital_count  question_count  negation_count  \
0                  0            115               0               0   

   fpp_count  capital_words  
0          0              0  
    id language main_img_url   published  \
0  0.0  english     NOT NULL  2020-04-21   

                                                text    title  type  \
0  President-elect Donald J. Trump s wife, Melani...  notitle  real   

                               uuid  word_count  average_word_count  \
0  584700e476e0d3c20731cb3d28e6ce2b         518                14.0   

   exclamation_count  capital_count  question_c

ValueError: ignored